### Encrypting Text

In [ ]:

def applyCodeBook(text,code_book):
    out=[]
    for c in text:
        if c==' ': # just keep spaces as spaces
            out.append(' ')
        elif okChar(c): # if not ok, we just skip it.
            out.append(code_book[c])
    s=""
    return s.join(out)

text="in the land of the blind, the one eyed person is king"
text=text.upper() # make upper case

encrypted_text=applyCodeBook(text,codebook)

print(f"Original Text: {text}")
print(f"Encrypted Text: {encrypted_text}")

### Decrypting Text

In [ ]:
def invertCodeBook(codeBook):
    inverted_book={ i[1]: i[0] for i in codeBook.items()  } # Switch the order to make inverse mapping
    return inverted_book

inverted_codebook=invertCodeBook(codebook)
decrypted_text=applyCodeBook(encrypted_text,inverted_codebook)
print(f"Decrypted Text: {decrypted_text}")

In [ ]:
def randSwapInCodeBook(codeBook):
    o=rnd.sample([k for k in codeBook],k=2)
    tmp=codeBook[o[0]]
    codeBook[o[0]]=codeBook[o[1]]
    codeBook[o[1]]=tmp

In [ ]:
num_swaps=2
for k in range(num_swaps):
    randSwapInCodeBook(inverted_codebook)

decrypted_text=applyCodeBook(encrypted_text,inverted_codebook)
print(f"Decrypted Text with corrupted codebook: \n{decrypted_text}")

In [ ]:
num_swaps=20
for k in range(num_swaps):
    randSwapInCodeBook(inverted_codebook)

decrypted_text=applyCodeBook(encrypted_text,inverted_codebook)
print(f"Decrypted Text with corrupted codebook: \n{decrypted_text}")

![image](MCMC_1_q.png)
![image](MCMC_1_ans.png)

![image](MCMC_2_q.png)
![image](MCMC_2_ans.png)

In [27]:
import numpy as np
import random as rnd

In [1]:
# These are some functions you might find useful.
# *cleanText*:  makes the string of characters all uppercase, 
# removes extra spaces, turns punctuation into spaces and 
# removes all other characters which are not A-Z including numbers
#
# *addCounts*: Sums up the frequency of each adjacent letter pair in the text. 
# This can be used to calculate the needed conditional probabilities.

def okChar(c): # make sure character is in the range we expect. Just a double check.
    if ((('A' <= c) and (c<='Z')) or (c==' ')):
        return True
    return False

def cleanText(data):
    data=data.upper()
    makeSpacesChar=[',','!','?',';','.',':']  # characters to change to spaces
    for i in range(10):
        makeSpacesChar.append(str(i))  # add all of the numbers to the list
    for c in makeSpacesChar:
        data=data.replace(c,' ')  # Replace each characters with a space
    data=' '.join(data.split()) # Remove extra spaces 
    onlyGoodChar=[c for c in data if ((c<='Z') and (c>='A')) or (c==' ')] # Remove all characters who are not A-Z or Space
    data=''.join(onlyGoodChar)
    return data

def addCounts(pairDictionary,data): 
    # counts the number of times c1 followed by c2 appears in text 
    # notice that keys that don't appear are missing. You should understand 
    # those to be zero counts 
    for i in range(len(data)-1):
        c1=data[i]
        c2=data[i+1]
        if  ( okChar(c1) and okChar(c2)) :
            key=(c1,c2)
            if key in pairDictionary: # Check to see if key=(c1,c2) is already there
                pairDictionary[key]+=1 # if it is add 
            else:
                pairDictionary[key]=1

def addLetterCounts(letterDictionary,data):
    for c in data:
        if okChar(c):
            if c in letterDictionary:
                letterDictionary[c]+=1
            else:
                letterDictionary[c]=1


In [2]:
# Add put whatever plan text files you want to make you frequency table out of
fileNames=["emma.txt","heart_of_darkness.txt","roger_ackroyd.txt",
           "the_war_of_the_worlds.txt","the_jungle.txt","THE_PROBLEMS_OF_PHILOSOPHY.txt",
           "The_Sun_Also_Rises.txt","the_war_of_the_worlds.txt","The_Souls_of_Black_Folk.txt",
           "Walden.txt"
           ]

pairDictionary={} # initialize empty pair dictionary

letterDictionary={} # for each letter count the number of times it appears

directory="Data/"
for fileName in fileNames: #cycle over the file names
    with open(directory+fileName, 'r') as file:
        data = file.read().replace('\n', '')
    data=cleanText(data)
    addCounts(pairDictionary,data) # aff the counts for the current file loaded
    addLetterCounts(letterDictionary,data) # adding counts of data

file_tag = open("encoded.txt", "r") # read in encoded message
encoded_text=file_tag.read()
file_tag.close()

In [7]:
print(letterDictionary['M'])
print(pairDictionary[('Q','U')])

95194
3675


In [13]:
# calculating conditional probabilities

def calculateConditionalProbailities(pairDictionary, letterDictionary):
    conditionalProbDictionary={} # initialize empty dictionary

    # generate all possible pairs of letters
    pairList = []
    for i in range(26):
        for j in range(26):
            pairList.append((chr(i+65), chr(j+65)))

    # appending ' ' to the pairList
    for i in range(26):
        pairList.append((' ', chr(i+65)))
        pairList.append((chr(i+65), ' '))


    for key in pairList:
        if key in pairDictionary:
            post = key[0]
            pre = key[1]
            conditionalProbDictionary[key] = pairDictionary[key] / letterDictionary[pre]
        else:
            conditionalProbDictionary[key] = 0

    return conditionalProbDictionary
        


In [14]:
conditionalProbabilityDict = calculateConditionalProbailities(pairDictionary, letterDictionary)

print(conditionalProbabilityDict[('Q', ' ')], conditionalProbabilityDict[('Q', 'U')])

0 0.03467929905351464


In [24]:
def log_likelihood(text, conditionalProbabilityDict):
    running_log_likelihood = 0

    epsilon = np.exp(-16) # small P to avoid log(0)
    log_epsilon = np.log(epsilon)
    abs_log_epsilon = np.abs(log_epsilon)

    for i in range(0, len(text)-1):
        logProb = np.log(conditionalProbabilityDict[(text[i], text[i+1])])
        abs_logProb = np.abs(logProb)
        current_log_likelihood = np.min([abs_logProb, abs_log_epsilon])
        running_log_likelihood += current_log_likelihood
    
    return -1 * running_log_likelihood

In [28]:
# create a random codebook
def randomCodebook():
    alpha=[chr(c) for c in range(ord('A'),ord('A')+26)] # create a list with A to Z 
    alpha2=alpha.copy() # Start with every letter mapping to itself
    rnd.shuffle(alpha2) # shuffles the letter around to create a random permutation
    codeBook={ a:code for a,code in zip(alpha,alpha2) } # Make a dictionary
    return codeBook

codebook=randomCodebook()
# print(codebook)

for (a,b) in codebook.items():
    print(f'{a} -> {b}')

A -> H
B -> R
C -> V
D -> Y
E -> W
F -> D
G -> L
H -> G
I -> O
J -> T
K -> J
L -> C
M -> U
N -> X
O -> S
P -> F
Q -> K
R -> Q
S -> N
T -> M
U -> A
V -> E
W -> Z
X -> I
Y -> B
Z -> P


In [29]:
# apply the codebook to the encoded text

def applyCodeBook(text,code_book):
    out=[]
    for c in text:
        if c==' ': # just keep spaces as spaces
            out.append(' ')
        elif okChar(c): # if not ok, we just skip it.
            out.append(code_book[c])
    s=""
    return s.join(out)

text="in the land of the blind, the one eyed person is king"
text=text.upper() # make upper case

encrypted_text=applyCodeBook(text,codebook)

print(f"Original Text: {text}")
print(f"Encrypted Text: {encrypted_text}")

Original Text: IN THE LAND OF THE BLIND, THE ONE EYED PERSON IS KING
Encrypted Text: OX MGW CHXY SD MGW RCOXY MGW SXW WBWY FWQNSX ON JOXL


In [30]:
# swap two random letters in the codebook

In [53]:
# log_likelihood(text, conditionalProbabilityDict):

def metropolis_step(current_codebook, current_text, current_log_likelihood, conditionalProbabilityDict):
    proposed_codebook = current_codebook.copy()
    randSwapInCodeBook(proposed_codebook)

    proposed_text = applyCodeBook(encoded_text, proposed_codebook)

    # current_log_likelihood = log_likelihood(current_text, conditionalProbabilityDict)
    proposed_log_likelihood = log_likelihood(proposed_text, conditionalProbabilityDict)

    acceptance_ratio = np.exp(proposed_log_likelihood - current_log_likelihood)

    acceptance_probability = np.min([1, acceptance_ratio])

    if rnd.random() < acceptance_probability:
        return proposed_codebook, proposed_text, proposed_log_likelihood
    else:
        return current_codebook, current_text, current_log_likelihood

In [56]:
def runMetropolisAlgorithm(initialCodebook, initial_text, initial_log_likehood, steps=2000):

    directory="Data/"
    for fileName in fileNames: #cycle over the file names
        with open(directory+fileName, 'r') as file:
            data = file.read().replace('\n', '')
        data=cleanText(data)
        addCounts(pairDictionary,data) # aff the counts for the current file loaded
        addLetterCounts(letterDictionary,data) # adding counts of data

    file_tag = open("encoded.txt", "r") # read in encoded message
    encoded_text=file_tag.read()
    file_tag.close()

    # codebook=randomCodebook()
    codebook = initialCodebook
    # current_text = applyCodeBook(encoded_text, codebook)
    current_text = initial_text
    # current_log_likelihood = log_likelihood(current_text, conditionalProbabilityDict)
    current_log_likelihood = initial_log_likehood

    for i in range(steps):
        codebook, current_text, current_log_likelihood = metropolis_step(codebook, current_text, current_log_likelihood, conditionalProbabilityDict)
        print(f'Iteration {i}: {current_log_likelihood}')
    
    #show decoded text
    return codebook

In [57]:
directory="Data/"
for fileName in fileNames: #cycle over the file names
    with open(directory+fileName, 'r') as file:
        data = file.read().replace('\n', '')
    data=cleanText(data)
    addCounts(pairDictionary,data) # aff the counts for the current file loaded
    addLetterCounts(letterDictionary,data) # adding counts of data

file_tag = open("encoded.txt", "r") # read in encoded message
encoded_text=file_tag.read()
file_tag.close()

new = runMetropolisAlgorithm()

decrypt = applyCodeBook(encoded_text,new)

print(f"Encrypted Text: {decrypt}")


/var/folders/91/nz1r31690rb_44hz9hkx_3rm0000gn/T/ipykernel_32681/930857222.py:9: RuntimeWarning: divide by zero encountered in log
  logProb = np.log(conditionalProbabilityDict[(text[i], text[i+1])])


Iteration 0: -31724.26591229292
Iteration 1: -31724.26591229292
Iteration 2: -31724.26591229292
Iteration 3: -31724.26591229292
Iteration 4: -31724.26591229292
Iteration 5: -31724.26591229292
Iteration 6: -31724.26591229292
Iteration 7: -31724.26591229292
Iteration 8: -31724.26591229292
Iteration 9: -31369.637166248893
Iteration 10: -31369.637166248893
Iteration 11: -31369.637166248893
Iteration 12: -31369.637166248893
Iteration 13: -30887.635117944894
Iteration 14: -30852.0436151384
Iteration 15: -30852.0436151384
Iteration 16: -30852.0436151384
Iteration 17: -30852.0436151384
Iteration 18: -30852.0436151384
Iteration 19: -30852.0436151384
Iteration 20: -30637.256791506778
Iteration 21: -30016.05955826841
Iteration 22: -29893.19580116626
Iteration 23: -29731.398900422584
Iteration 24: -29731.398900422584
Iteration 25: -29731.398900422584
Iteration 26: -29731.398900422584
Iteration 27: -29731.398900422584
Iteration 28: -29731.398900422584
Iteration 29: -29731.398900422584
Iteration 30:

/var/folders/91/nz1r31690rb_44hz9hkx_3rm0000gn/T/ipykernel_32681/2569695612.py:12: RuntimeWarning: overflow encountered in exp
  acceptance_ratio = np.exp(proposed_log_likelihood - current_log_likelihood)


Iteration 36: -28790.892212524974
Iteration 37: -28790.892212524974
Iteration 38: -28790.892212524974
Iteration 39: -28790.892212524974
Iteration 40: -28790.892212524974
Iteration 41: -28790.892212524974
Iteration 42: -28790.892212524974
Iteration 43: -28750.339507303255
Iteration 44: -28525.910021457228
Iteration 45: -28525.910021457228
Iteration 46: -28525.910021457228
Iteration 47: -28525.910021457228
Iteration 48: -28525.910021457228
Iteration 49: -28525.910021457228
Iteration 50: -28525.910021457228
Iteration 51: -28525.910021457228
Iteration 52: -28525.910021457228
Iteration 53: -28525.910021457228
Iteration 54: -27916.601079584594
Iteration 55: -27916.601079584594
Iteration 56: -27916.601079584594
Iteration 57: -27916.601079584594
Iteration 58: -27916.601079584594
Iteration 59: -27916.601079584594
Iteration 60: -27916.601079584594
Iteration 61: -27916.601079584594
Iteration 62: -27916.601079584594
Iteration 63: -27916.601079584594
Iteration 64: -27916.601079584594
Iteration 65: 

In [ ]:
# continue running the algorithm from current state
